In [0]:
%sql
-- Definição do catálogo e schema utilizados na camada Silver.

USE CATALOG mvp;
USE SCHEMA silver;

In [0]:
# Leitura da Delta Table criada na Camada Bronze, que contém os dados brutos padronizados

df_voos_bronze = spark.table("mvp.bronze.voos")

In [0]:
# Conforme definido no objetivo do projeto, filtraremos somente os vôos mistos em que todos os aeroportos envolvidos estejam situados simultaneamente em
# território brasileiro, sendo definido com a legenda "N", conforme descrição de variáveis disponibilizada pela ANAC.

from pyspark.sql.functions import col

df_voos_filtrado = (
    df_voos_bronze
    .filter(col("codigo_tipo_linha") == "N")
)


In [0]:
# Conversão dos campos das datas de string para timestamp
# Utilização do try_to_timestamp combinado com coalesce para garantir a conversão
# independente do formato e preservando os inválidos como NULL

from pyspark.sql.functions import coalesce, expr

df_voos_tempo = (
    df_voos_filtrado
    .withColumn(
        "partida_real_ts",
        coalesce(
            expr("try_to_timestamp(partida_real, 'dd/MM/yyyy HH:mm')"),
            expr("try_to_timestamp(partida_real, 'yyyy-MM-dd HH:mm:ss')")
        )
    )
    .withColumn(
        "chegada_real_ts",
        coalesce(
            expr("try_to_timestamp(chegada_real, 'dd/MM/yyyy HH:mm')"),
            expr("try_to_timestamp(chegada_real, 'yyyy-MM-dd HH:mm:ss')")
        )
    )
)

In [0]:
# Definição das colunas relevantes para a Camada Silver

df_voos_silver = df_voos_tempo.select(
    "icao_empresa_aerea",
    "numero_voo",
    "icao_aerodromo_origem",
    "icao_aerodromo_destino",
    "situacao_voo",
    "partida_real_ts",
    "chegada_real_ts"
)


In [0]:
# Criação de flag para indicação de voôs realizados
# A flag assume valor 1 para voos com situação "REALIZADO" e 0 caso contrário

from pyspark.sql.functions import when

df_voos_silver = (
    df_voos_silver
    .withColumn(
        "flag_realizado",
        when(col("situacao_voo") == "REALIZADO", 1).otherwise(0)
    )
)


In [0]:
# Criação das colunas de ano e mês do voo com base no timestamp de partida

from pyspark.sql.functions import year, month

df_voos_silver = (
    df_voos_silver
    .withColumn("ano_voo", year(col("partida_real_ts")))
    .withColumn("mes_voo", month(col("partida_real_ts")))
)

In [0]:
# Criação da coluna de estação do ano com base no mês de partida

from pyspark.sql.functions import when

df_voos_silver = (
    df_voos_silver
    .withColumn(
        "estacao_ano",
        when(col("mes_voo").isin(12,1,2), "Verao")
        .when(col("mes_voo").isin(3,4,5), "Outono")
        .when(col("mes_voo").isin(6,7,8), "Inverno")
        .when(col("mes_voo").isin(9,10,11), "Primavera")
        .otherwise(None)
    )
)

In [0]:
# Criação da tabela Silver em formato delta para armazenar os dados

(
    df_voos_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable("mvp.silver.voos")
)

In [0]:
%sql
-- Consulta de validação da Camada Silver trazendo totais de registros, companhias e aeroportos

SELECT
    COUNT(*) AS total_registros,
    COUNT(DISTINCT icao_empresa_aerea) AS total_companhias,
    COUNT(DISTINCT icao_aerodromo_origem) AS total_aeroportos_origem,
    COUNT(DISTINCT icao_aerodromo_destino) AS total_aeroportos_destino
FROM mvp.silver.voos;

total_registros,total_companhias,total_aeroportos_origem,total_aeroportos_destino
796930,15,208,209


In [0]:
%sql
-- Análise da distribuição de voos por situação após tratamento realizado na Camada Silver

SELECT situacao_voo, COUNT(*) AS quantidade
FROM mvp.silver.voos
GROUP BY situacao_voo;

situacao_voo,quantidade
REALIZADO,772412
CANCELADO,23137
NÃO INFORMADO,1381


In [0]:
%sql
-- Avalia a quantidade de registros sem timestamp de partida real,
-- permitindo confirmar se os valores nulos estão associados a voos não realizados.

SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN partida_real_ts IS NULL THEN 1 ELSE 0 END) AS nulos_partida
FROM mvp.silver.voos;

total,nulos_partida
796930,24518


In [0]:
%sql
-- Consulta exploratória da Camada Silver

SELECT * 
FROM mvp.silver.voos
LIMIT 100;

icao_empresa_aerea,numero_voo,icao_aerodromo_origem,icao_aerodromo_destino,situacao_voo,partida_real_ts,chegada_real_ts,flag_realizado,ano_voo,mes_voo,estacao_ano
AZU,4141,SBKP,SBSV,REALIZADO,2024-01-21T14:56:00.000Z,2024-01-21T17:15:00.000Z,1,2024,1,Verao
AZU,4141,SBKP,SBSV,REALIZADO,2024-01-22T16:39:00.000Z,2024-01-22T18:49:00.000Z,1,2024,1,Verao
AZU,4141,SBKP,SBSV,REALIZADO,2024-01-24T14:56:00.000Z,2024-01-24T17:13:00.000Z,1,2024,1,Verao
AZU,4141,SBKP,SBSV,REALIZADO,2024-01-26T15:00:00.000Z,2024-01-26T17:25:00.000Z,1,2024,1,Verao
AZU,4141,SBKP,SBSV,REALIZADO,2024-01-29T15:37:00.000Z,2024-01-29T18:14:00.000Z,1,2024,1,Verao
AZU,4141,SBKP,SBSV,REALIZADO,2024-01-31T15:35:00.000Z,2024-01-31T17:58:00.000Z,1,2024,1,Verao
AZU,4142,SBCF,SBCF,REALIZADO,2024-01-30T17:43:00.000Z,2024-01-30T17:55:00.000Z,1,2024,1,Verao
AZU,4142,SBCF,SBIP,REALIZADO,2024-01-01T17:52:00.000Z,2024-01-01T18:33:00.000Z,1,2024,1,Verao
AZU,4142,SBCF,SBIP,REALIZADO,2024-01-02T17:42:00.000Z,2024-01-02T18:28:00.000Z,1,2024,1,Verao
AZU,4142,SBCF,SBIP,REALIZADO,2024-01-03T18:04:00.000Z,2024-01-03T18:43:00.000Z,1,2024,1,Verao


In [0]:
%sql
-- Consulta trazendo todas as companhias que operaram voos domésticos realizados em 2024
-- para embasar a criação da tabela Silver de Companhias

SELECT icao_empresa_aerea AS companhias, COUNT(*) AS total_voos
FROM mvp.silver.voos
WHERE situacao_voo = 'REALIZADO'
GROUP BY icao_empresa_aerea
ORDER BY total_voos DESC;

companhias,total_voos
AZU,290158
TAM,240808
GLO,199242
ACN,20560
PTB,15602
PAM,3889
ABJ,1084
SID,539
CQB,261
MWM,132
